<a href="https://colab.research.google.com/github/mitali72/Generating-aligned-transcripts/blob/main/silero_vad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#VAD

In [ ]:
#@title Install and Import Dependencies

# this assumes that you have a relevant version of PyTorch installed
!pip install -q torchaudio

SAMPLING_RATE = 16000

import torch
torch.set_num_threads(1)

from IPython.display import Audio
from pprint import pprint
# download example
torch.hub.download_url_to_file('https://models.silero.ai/vad_models/en.wav', 'en_example.wav')

  0%|          | 0.00/1.83M [00:00<?, ?B/s]

In [ ]:
USE_ONNX = False # change this to True if you want to test onnx model
if USE_ONNX:
    !pip install -q onnxruntime

# Tested on version 3.1 of silero only, had a bug so fixed and forked on my account

model, utils = torch.hub.load(repo_or_dir='mitali72/silero-vad:v3.1.1',
                              model='silero_vad',
                              force_reload=True)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

Downloading: "https://github.com/mitali72/silero-vad/zipball/v3.1.1" to /root/.cache/torch/hub/v3.1.1.zip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/BTP/NewsOnAir.zip -d /content/NewsOnAir

Archive:  /content/drive/MyDrive/BTP/NewsOnAir.zip
   creating: /content/NewsOnAir/Gujarati/
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-1320-1330-202262213512.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-1320-1330-202274133759.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-0745-0755-20225481235.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-0745-0755-2022819250.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-0745-0755-20224148830.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-1320-1330-202276134830.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-1950-2000-20228520538.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-0745-0755-202282184951.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-0745-0755-202282781351.mp3  
  inflating: /content/NewsOnAir/Gujarati/NSD-Gujarati-Gujarati-1950-2000-20228921348

## Full Audio

**Speech timestapms from full audio**

In [ ]:
!pip install pydub torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# display(Audio("https://newsonair.gov.in/writereaddata/Bulletins_Audio/NSD/2022/Aug/NSD-Gujarati-Gujarati-0745-0755-202281981355.mp3"))
from IPython.display import Audio
from IPython.display import display
from pydub import AudioSegment
import os

# Convert all mp3 files to wav with 16kHz sampling rate
mp3_folder = "/content/NewsOnAir/"
wav_folder = "/content/NewsOnAirWav"
!mkdir "$wav_folder"
for root, dirs, files in os.walk(mp3_folder):
  for audio_file in files:
    audio_path = os.path.join(root,audio_file)
    sound = AudioSegment.from_mp3(audio_path)
    sound = sound.set_frame_rate(16000)
    sound = sound.set_channels(1)
    wav_dir = os.path.join("/content/NewsOnAirWav/",audio_path.split("/")[3])
    if not os.path.exists(wav_dir):
      os.mkdir(wav_dir)

    sound.export(os.path.join(wav_dir,audio_path.split("/")[-1][:-3]+"wav"), format="wav")


In [ ]:
wav = read_audio('/content/NewsOnAirWav/Marathi/NSD-Marathi-Marathi-1330-1340-2022820165427.wav', sampling_rate=SAMPLING_RATE)
# get speech timestamps from full audio file
speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=SAMPLING_RATE, threshold = 0.6)
# pprint(speech_timestamps)

In [ ]:
from pathlib import Path
import torchaudio

# Save Silero VAD chunks
for root, dirs, files in os.walk(wav_folder):
  for audio_file in files:
    audio_path = os.path.join(root,audio_file)
    wav = read_audio(audio_path)
    speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=SAMPLING_RATE, threshold = 0.6)

    audio_name = Path(audio_path).stem
    chunk_path = os.path.join('/content/NewsOnAirVAD',audio_path.split("/")[3],audio_name)
    Path(chunk_path).mkdir(parents = True,exist_ok=True)
    for i,vad_sample in enumerate(speech_timestamps):
      torchaudio.save(os.path.join(chunk_path,'chunk-%002d.wav' % (i,)), wav[vad_sample['start']:vad_sample['end']].unsqueeze(0), 16000)



In [ ]:
# from pathlib import Path
# Save single file
# # wav_file = "/content/NewsOnAirWav/Gujarati/NSD-Gujarati-Gujarati-1320-1330-202282814554.wav"
# wav_file = "/content/NewsOnAirWav/Marathi/NSD-Marathi-Marathi-1330-1340-2022820165427.wav"

# import torchaudio
# audio_name = Path(wav_file).stem
# chunk_path = os.path.join('/content/NewsOnAirVAD',wav_file.split("/")[3],audio_name)
# Path(chunk_path).mkdir(parents = True,exist_ok=True)
# for i,vad_sample in enumerate(speech_timestamps):
#   torchaudio.save(os.path.join(chunk_path,'chunk-%002d.wav' % (i,)), wav[vad_sample['start']:vad_sample['end']].unsqueeze(0), 16000)

In [ ]:
# merge all speech chunks to one audio
# save_audio('only_speech_mar1330.wav',collect_chunks(speech_timestamps, wav), sampling_rate=SAMPLING_RATE)

In [ ]:
import numpy as np

def wada_snr(wav):
    # Direct blind estimation of the SNR of a speech signal.
    #
    # Paper on WADA SNR:
    #   http://www.cs.cmu.edu/~robust/Papers/KimSternIS08.pdf
    #
    # This function was adapted from this matlab code:
    #   https://labrosa.ee.columbia.edu/projects/snreval/#9

    # init
    eps = 1e-10
    # next 2 lines define a fancy curve derived from a gamma distribution -- see paper
    db_vals = np.arange(-20, 101)
    g_vals = np.array([0.40974774, 0.40986926, 0.40998566, 0.40969089, 0.40986186, 0.40999006, 0.41027138, 0.41052627, 0.41101024, 0.41143264, 0.41231718, 0.41337272, 0.41526426, 0.4178192 , 0.42077252, 0.42452799, 0.42918886, 0.43510373, 0.44234195, 0.45161485, 0.46221153, 0.47491647, 0.48883809, 0.50509236, 0.52353709, 0.54372088, 0.56532427, 0.58847532, 0.61346212, 0.63954496, 0.66750818, 0.69583724, 0.72454762, 0.75414799, 0.78323148, 0.81240985, 0.84219775, 0.87166406, 0.90030504, 0.92880418, 0.95655449, 0.9835349 , 1.01047155, 1.0362095 , 1.06136425, 1.08579312, 1.1094819 , 1.13277995, 1.15472826, 1.17627308, 1.19703503, 1.21671694, 1.23535898, 1.25364313, 1.27103891, 1.28718029, 1.30302865, 1.31839527, 1.33294817, 1.34700935, 1.3605727 , 1.37345513, 1.38577122, 1.39733504, 1.40856397, 1.41959619, 1.42983624, 1.43958467, 1.44902176, 1.45804831, 1.46669568, 1.47486938, 1.48269965, 1.49034339, 1.49748214, 1.50435106, 1.51076426, 1.51698915, 1.5229097 , 1.528578  , 1.53389835, 1.5391211 , 1.5439065 , 1.54858517, 1.55310776, 1.55744391, 1.56164927, 1.56566348, 1.56938671, 1.57307767, 1.57654764, 1.57980083, 1.58304129, 1.58602496, 1.58880681, 1.59162477, 1.5941969 , 1.59693155, 1.599446  , 1.60185011, 1.60408668, 1.60627134, 1.60826199, 1.61004547, 1.61192472, 1.61369656, 1.61534074, 1.61688905, 1.61838916, 1.61985374, 1.62135878, 1.62268119, 1.62390423, 1.62513143, 1.62632463, 1.6274027 , 1.62842767, 1.62945532, 1.6303307 , 1.63128026, 1.63204102])

    # peak normalize, get magnitude, clip lower bound
    wav = np.array(wav)
    wav = wav / abs(wav).max()
    abs_wav = abs(wav)
    abs_wav[abs_wav < eps] = eps

    # calcuate statistics
    # E[|z|]
    v1 = max(eps, abs_wav.mean())
    # E[log|z|]
    v2 = np.log(abs_wav).mean()
    # log(E[|z|]) - E[log(|z|)]
    v3 = np.log(v1) - v2

    # table interpolation
    wav_snr_idx = None
    if any(g_vals < v3):
        wav_snr_idx = np.where(g_vals < v3)[0].max()
    # handle edge cases or interpolate
    if wav_snr_idx is None:
        wav_snr = db_vals[0]
    elif wav_snr_idx == len(db_vals) - 1:
        wav_snr = db_vals[-1]
    else:
        wav_snr = db_vals[wav_snr_idx] + \
            (v3-g_vals[wav_snr_idx]) / (g_vals[wav_snr_idx+1] - \
            g_vals[wav_snr_idx]) * (db_vals[wav_snr_idx+1] - db_vals[wav_snr_idx])

    # Calculate SNR
    dEng = sum(wav**2)
    dFactor = 10**(wav_snr / 10)
    dNoiseEng = dEng / (1 + dFactor) # Noise energy
    dSigEng = dEng * dFactor / (1 + dFactor) # Signal energy
    snr = 10 * np.log10(dSigEng / dNoiseEng)

    return wav_snr

In [ ]:
from pathlib import Path
import numpy as np
from scipy.io import wavfile
import csv


chunk_fields = ["audio_name", "chunk_id", "snr", "len"]
vad_folder = "/content/NewsOnAirVAD/"

f = open('/content/drive/MyDrive/BTP/chunk_info_silero_new.csv', 'w')
writer = csv.DictWriter(f, fieldnames = chunk_fields)
writer.writeheader()
for root, dirs, files in os.walk(vad_folder):
  chunk_info = {}
  
  for audio_file in sorted(files):
    
    audio_path = os.path.join(root,audio_file)
    _,wav = wavfile.read(audio_path)
    chunk_snr = wada_snr(wav)

    chunk_info["audio_name"] = audio_path.split("/")[-2]
    chunk_info["chunk_id"] = Path(audio_file).stem[-2:]
    chunk_info["snr"] = chunk_snr
    chunk_info["len"] = len(wav)/16000
 
    writer.writerow(chunk_info)

f.close()

# **Whisper**

In [ ]:
!pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-8jlndzgu
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-8jlndzgu
     |████████████████████████████████| 5.5 MB 32.5 MB/s 
     |████████████████████████████████| 163 kB 74.0 MB/s 
     |████████████████████████████████| 7.6 MB 54.8 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175217 sha256=ee1752041982bb58f8a5fc8b124d6754317d9ad4913c7aaeb1292909473d4319
  Stored in directory: /tmp/pip-ephem-wheel-cache-ukjdoltj/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper


In [ ]:
from IPython.display import Audio

In [ ]:
test_chunk = "/content/NewsOnAirVADChunks/Marathi/NSD-Marathi-Marathi-1330-1340-2022820165427/chunk-69.wav"
wn = Audio(test_chunk)
display(wn)
!whisper $test_chunk --language Marathi  

[00:00.000 --> 00:03.940]  राज्यात कल कोविदे कुनिष्या 2020 पन्षेंषी नवेर्गुनाईची नोध जली
[00:04.240 --> 00:06.600]  2020 सदादी स्रुणा कुरना मुक्ता जले
[00:06.600 --> 00:08.600]  तर पाच रुगनान साम्रिक्तिव साला


In [ ]:
stt_path = "/content/drive/MyDrive/BTP/NSD-Marathi-Marathi-1330-1340-2022820165427_stt2"

In [ ]:
import os
import subprocess
from pathlib import Path
import numpy as np
from scipy.io import wavfile
from tqdm import tqdm

# Generate text using whisper for all VAD chunks
vad_folder = "/content/NewsOnAirVADChunks/"
for root, dirs, files in os.walk(vad_folder):
  chunk_info = {}
  
  for i,audio_file in tqdm(enumerate(sorted(files))):
    
    audio_path = os.path.join(root,audio_file)
    _,wav = wavfile.read(audio_path)
    chunk_snr = wada_snr(wav)
    if chunk_snr>=15 and chunk_snr<=60:
      subprocess.call(f"whisper {audio_path} --language Marathi --compression_ratio_threshold 1.8 --no_speech_threshold 0.4 --condition_on_previous_text False",shell=True,cwd=stt_path)
      # os.system(f"!whisper {audio_path} --language Marathi --compression_ratio_threshold 1.8 --no_speech_threshold 0.4 --condition_on_previous_text False")
    # embed = encoder.embed_utterance(preprocess_wav(audio_path))

0it [00:00, ?it/s]
0it [00:00, ?it/s]
89it [22:25, 15.12s/it]


In [ ]:
!pip install pip setuptools wheel
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !mkdir /content/drive/MyDrive/BTP/NSD-Marathi-Marathi-1330-1340-2022820165427_stt
import os
stt_path = "/content/drive/MyDrive/BTP/NSD-Marathi-Marathi-1330-1340-2022820165427_stt2"
stt_files = os.listdir(stt_path)
stt_files = [file for file in stt_files if file.endswith(".txt")]
# print(sorted(stt_files))
trans_txt = []
for stt_file in stt_files:
  phrase = ""
  with open(os.path.join(stt_path,stt_file)) as f_in:
    lines = (line.rstrip() for line in f_in)
    for line in lines:
      if line:
        phrase+=line 

    trans_txt.append(phrase)

print(trans_txt)

['अखेश्वानी मूमभी', 'अदेश नाएक', 'राश्ट्रिया बात्म्या दिता हे', 'तो ड़ाद बाद में', 'इंदेशातल्या बैंका प्रशासक्या दूष्त्या सक्शम होत अस्लया', 'अदिक सुदारना अवश्यक आज्लेज़ अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्यक अवश्य', 'रीजरो बेंकेचा अप्यास अहवालात मत्ला है', 'बार्तिया बेंकान सो प्रशासन', 'कर्यक्शमता अने सुबोदता या बढब तरीजरो बेंके चा हा अवाल असुनगेल्या कही वर्षाद भर्तात्या बेंकानी प्रशासा की आप प्रमनाकान सा पालन के ला अस्लग अवाल प्रशासा की आप प्रमनाकान सा पालन के ला अस्लग प्रशासा की आप प्रशासा की आप प्रमनाकान सा पालन के ला अस्लग प्रशासा की आप प्रशासा की आप प्', 'तरी सा ड्यामात्रा या चास्तोर', 'पूरे सा योगे नास्लाचा या अवालात मतला है', 'बेंकान जा दिर्गखालिन कर्यगाडा जोखिम तारनेक्र ता शाश्वत लबाचा उपाएजुना ना चालना दे़ा अवश्यक अस्लय ज़ा या अह्वालात मतला है', '2018-9-20122018-9-2012', 'भारद्या बैंकेंग उद्ध्योगाची स्तितिट सानली होती', 'मात्र 2013-14 नत्र याद माल्मत्त, गुणुवत्त, अन नफ्याचा दु

In [ ]:
orig_txt = ""
with open('/content/drive/MyDrive/BTP/orig.txt') as f_in:
    lines = (line.rstrip() for line in f_in) # All lines including the blank ones
    for line in lines:
      if line:
        orig_txt+=line  # Non-blank lines in a list

print(orig_txt)

आकाशवाणी मुंबईराष्ट्रीय बातमीपत्र - दु. १.३०वाजता२० ऑगस्ट २०२२ - शनिवारठळक बातम्यादेशातल्या बँका प्रशासकीयदृष्ट्या सक्षम होत असल्या तरी सामाजिक दृष्टिकोनातून अधिक सुधारणा आवश्यक असल्याचारिझर्व बँकेचा अहवालमाजी प्रधानमंत्री दिवंगत राजीव गांधी यांच्या जयंतीनिमित्त देशाची आदरांजलीमहाराष्ट्रात जखमी गोविंदांच्या उपचारांबाबतचे शासन निर्णय जारी/ मुंबईत काल दिवसभरात १११ गोविंदा जखमीराजस्थानात पाली इथं रस्ते अपघातात ७ जण ठार, तर २४ हून अधिक जखमीझिंबाब्वेबरोबर दुसऱ्या एकदिवसीय क्रिकेट सामन्यात नाणेफेक जिंकून भारताचा क्षेत्ररक्षणाचा निर्णयदेशातल्या बँका प्रशासकीयदृष्टया सक्षम होत असल्या तरी सामाजिक दृष्टिकोनातून अधिक सुधारणा आवश्यक असल्याचंभारतीय रिझर्व बँकेच्या अभ्यास अहवालात म्हटलं आहे. भारतीय बँकांचं 'प्रशासन, कार्यक्षमता आणि सुबोधता' याबाबत रिझर्वबँकेचा हा अहवाल असून गेल्या काही वर्षात भारतातल्या बँकांनी प्रशासकीय प्रमाणकांच पालन केलं असलं तरी सध्या मात्र याचास्तर पुरेसा योग्य नसल्याचं या अहवालात म्हटलं आहे. बँकांच्या दीर्घकालीन कार्यकाळात जोखीम टाळण्याकरता शाश्वत लाभाच्याउपाययोजनांना चालना द

In [ ]:
import spacy

from spacy.lang.mr import Marathi
nlp = Marathi()  # use directly

# Create the pipeline 'sentencizer' component
# sbd = nlp.create_pipe('sentencizer')

# Add the component to the pipeline
nlp.add_pipe('sentencizer')

#  "nlp" Object is used to create documents with linguistic annotations.
doc = nlp(orig_txt)

# create list of sentence tokens
orig_sent = []
for sent in doc.sents:
  orig_sent.append(sent.text)

print(orig_sent)
# orig_tokenized = [token.text for token in doc]
# print(orig_tokenized)

['आकाशवाणी मुंबईराष्ट्रीय बातमीपत्र - दु.', '१.३०वाजता२० ऑगस्ट २०२२ - शनिवारठळक बातम्यादेशातल्या बँका प्रशासकीयदृष्ट्या सक्षम होत असल्या तरी सामाजिक दृष्टिकोनातून अधिक सुधारणा आवश्यक असल्याचारिझर्व बँकेचा अहवालमाजी प्रधानमंत्री दिवंगत राजीव गांधी यांच्या जयंतीनिमित्त देशाची आदरांजलीमहाराष्ट्रात जखमी गोविंदांच्या उपचारांबाबतचे शासन निर्णय जारी/ मुंबईत काल दिवसभरात १११ गोविंदा जखमीराजस्थानात पाली इथं रस्ते अपघातात ७ जण ठार, तर २४ हून अधिक जखमीझिंबाब्वेबरोबर दुसऱ्या एकदिवसीय क्रिकेट सामन्यात नाणेफेक जिंकून भारताचा क्षेत्ररक्षणाचा निर्णयदेशातल्या बँका प्रशासकीयदृष्टया सक्षम होत असल्या तरी सामाजिक दृष्टिकोनातून अधिक सुधारणा आवश्यक असल्याचंभारतीय रिझर्व बँकेच्या अभ्यास अहवालात म्हटलं आहे.', "भारतीय बँकांचं 'प्रशासन, कार्यक्षमता आणि सुबोधता' याबाबत रिझर्वबँकेचा हा अहवाल असून गेल्या काही वर्षात भारतातल्या बँकांनी प्रशासकीय प्रमाणकांच पालन केलं असलं तरी सध्या मात्र याचास्तर पुरेसा योग्य नसल्याचं या अहवालात म्हटलं आहे.", 'बँकांच्या दीर्घकालीन कार्यकाळात जोखीम टाळण्याकरता शाश्वत लाभाच्याउपाययोजना